### <b>3.3 PREPROCESSAMENT I ANÀLISI DE DADES</b>

### <b>3.3.1 Integració i neteja de dades</b>

#### <b>3.3.1.1 Càrrega i verificació inicial de les dades</b>

In [17]:
# ============================================================
# 3.3.1.1 Càrrega i verificació inicial de les dades
# ============================================================
# Objectiu:
#   - Carrego el dataset principal (CSV) i el diccionari de variables (XLSX)
#   - Verifico dimensions, estructura, tipus de dades, duplicats i valors nuls
#   - Comprovo la coherència entre les variables del dataset i les del diccionari
#   - Genero un primer resum inicial que sigui auditable
#
# Nota:
#   Aquest codi està pensat únicament per a la CÀRREGA i la VERIFICACIÓ INICIAL.
#   En aquest punt no faig cap imputació ni cap transformació de dades.
# ============================================================

import pandas as pd  # Importo pandas com a llibreria principal per a la manipulació de dades
import numpy as np   # Importo numpy per a operacions numèriques (tot i que aquí no s'utilitza directament)

# ------------------------------------------------------------
# 0) Rutes dels fitxers
# ------------------------------------------------------------
# Defineixo les rutes als fitxers d'entrada.
# Si canvia la ubicació dels fitxers, només cal modificar aquestes variables.
DATA_PATH = "Motor vehicle insurance data.csv"
DICT_PATH = "Descriptive of the variables.xlsx"

# ------------------------------------------------------------
# 1) Càrrega robusta del CSV
# ------------------------------------------------------------
# Llegeixo el fitxer CSV utilitzant detecció automàtica del separador.
# D'aquesta manera funciona tant si el separador és ',' com si és ';'.
df = pd.read_csv(DATA_PATH, sep=None, engine="python")
print("Dataset carregat correctament")

# ------------------------------------------------------------
# 2) Càrrega del diccionari de variables (Excel)
# ------------------------------------------------------------
# Carrego el fitxer Excel del diccionari per poder inspeccionar totes les fulles disponibles.
dict_xls = pd.ExcelFile(DICT_PATH)
print("\nFulls disponibles al diccionari:", dict_xls.sheet_names)

# Treballo amb la primera fulla del diccionari, que és on hi ha la descripció de les variables.
dict_df = pd.read_excel(DICT_PATH, sheet_name=dict_xls.sheet_names[0])
print("\nDiccionari carregat correctament")

# ------------------------------------------------------------
# 3) Verificació inicial del dataset
# ------------------------------------------------------------
# 3.1 Dimensions del dataset
# Comprovo el nombre total de files i columnes.
print("\nDimensions del dataset (files, columnes):", df.shape)

# 3.2 Mostra de dades
# Visualitzo les primeres 5 files per tenir una idea ràpida de l'estructura.
print("\nPrimeres 5 files del dataset:")
display(df.head())

# 3.3 Noms de les columnes
# Llisto totes les variables disponibles al dataset.
print("\nColumnes del dataset:")
print(df.columns.tolist())

# 3.4 Tipus de dades
# Reviso el tipus de dades assignat a cada columna.
print("\nTipus de dades per columna:")
print(df.dtypes)

# Distribució general dels tipus de dades
# Això em permet veure ràpidament si predominen variables numèriques, categòriques, etc.
print("\nDistribució de tipus:")
print(df.dtypes.value_counts())

# ------------------------------------------------------------
# 4) Coherència amb el diccionari de variables
# ------------------------------------------------------------
# Analitzo l'estructura del diccionari de variables.
print("\nDimensions del diccionari:", dict_df.shape)
print("\nPrimeres files del diccionari:")
display(dict_df.head())

# Extrec els noms de les variables del diccionari i del dataset.
# Els netejo de possibles espais en blanc per evitar errors de comparació.
dict_vars = set(dict_df["Variables"].astype(str).str.strip())
data_vars = set(df.columns.astype(str).str.strip())

# Identifico discrepàncies entre el diccionari i el dataset.
vars_in_dict_not_data = dict_vars - data_vars
vars_in_data_not_dict = data_vars - dict_vars

print("\nVariables al diccionari però no al dataset:", vars_in_dict_not_data)
print("Variables al dataset però no al diccionari:", vars_in_data_not_dict)

# ------------------------------------------------------------
# 5) Duplicats i estructura panel
# ------------------------------------------------------------
# 5.1 Duplicats exactes
# Comprovo si hi ha files completament duplicades.
n_dup_rows = df.duplicated().sum()
print("\nNombre de files duplicades exactes:", n_dup_rows)

# 5.2 Unicitat d'ID
# Analitzo quants IDs són únics, tenint en compte que espero repeticions per anualitats (estructura panel).
n_unique_ids = df["ID"].nunique()
pct_repeated = 1 - (n_unique_ids / len(df))
print("Nombre d'IDs únics:", n_unique_ids)
print("Percentatge d'IDs repetits (panel):", round(pct_repeated, 4))

# ------------------------------------------------------------
# 6) Taula de valors nuls
# ------------------------------------------------------------
# Calculo el nombre i el percentatge de valors nuls per variable.
missing_count = df.isna().sum()
missing_pct = (missing_count / len(df) * 100).round(2)

# Creo una taula resum ordenada per nombre de valors nuls.
missing_table = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": missing_pct
}).sort_values("missing_count", ascending=False)

print("\nTaula de valors nuls (top 10):")
display(missing_table.head(10))

# Identifico variables amb més del 10% de valors nuls.
high_missing_vars = missing_table[missing_table["missing_pct"] > 10].index.tolist()
print("Variables amb >10% nuls:", high_missing_vars)

# ------------------------------------------------------------
# 7) Rangs bàsics de variables numèriques
# ------------------------------------------------------------
# Selecciono només les columnes numèriques.
num_cols = df.select_dtypes(include=[np.number]).columns

# Calculo els valors mínims i màxims per detectar possibles valors anòmals.
ranges = df[num_cols].agg(["min", "max"]).T
print("\nRangs mínim i màxim de variables numèriques:")
display(ranges)

# ------------------------------------------------------------
# 8) Resum executiu inicial
# ------------------------------------------------------------
# Creo un petit resum amb els principals indicadors obtinguts fins ara.
summary = {
    "Registres": len(df),
    "Variables": df.shape[1],
    "Duplicats exactes": int(n_dup_rows),
    "Variables amb >10% nuls": high_missing_vars
}

print("\n--- RESUM INICIAL ---")
for k, v in summary.items():
    print(f"{k}: {v}")

# Deixo carregats els objectes principals per als següents subapartats de l'anàlisi:
# df       -> dataset principal
# dict_df  -> diccionari de variables


Dataset carregat correctament

Fulls disponibles al diccionari: ['Motor vehicle insurance data', 'sample type claim']

Diccionari carregat correctament

Dimensions del dataset (files, columnes): (105555, 30)

Primeres 5 files del dataset:


,ID,Date_start_contract,Date_last_renewal,Date_next_renewal,Date_birth,Date_driving_licence,Distribution_channel,Seniority,Policies_in_force,Max_policies,...,Area,Second_driver,Year_matriculation,Power,Cylinder_capacity,Value_vehicle,N_doors,Type_fuel,Length,Weight
0,1,05/11/2015,05/11/2015,05/11/2016,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
1,1,05/11/2015,05/11/2016,05/11/2017,15/04/1956,20/03/1976,0,4,1,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
2,1,05/11/2015,05/11/2017,05/11/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
3,1,05/11/2015,05/11/2018,05/11/2019,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190
4,2,26/09/2017,26/09/2017,26/09/2018,15/04/1956,20/03/1976,0,4,2,2,...,0,0,2004,80,599,7068.0,0,P,NaN,190



Columnes del dataset:
['ID', 'Date_start_contract', 'Date_last_renewal', 'Date_next_renewal', 'Date_birth', 'Date_driving_licence', 'Distribution_channel', 'Seniority', 'Policies_in_force', 'Max_policies', 'Max_products', 'Lapse', 'Date_lapse', 'Payment', 'Premium', 'Cost_claims_year', 'N_claims_year', 'N_claims_history', 'R_Claims_history', 'Type_risk', 'Area', 'Second_driver', 'Year_matriculation', 'Power', 'Cylinder_capacity', 'Value_vehicle', 'N_doors', 'Type_fuel', 'Length', 'Weight']

Tipus de dades per columna:
ID                        int64
Date_start_contract      object
Date_last_renewal        object
Date_next_renewal        object
Date_birth               object
Date_driving_licence     object
Distribution_channel      int64
Seniority                 int64
Policies_in_force         int64
Max_policies              int64
Max_products              int64
Lapse                     int64
Date_lapse               object
Payment                   int64
Premium                 flo

,Variables,Description
0,ID,Internal identification number assigned to eac...
1,Date_start _contract,Start date of the policyholder's contract (DD/...
2,Date_last_renewal,Date of last contract renewal (DD/MM/YYYY).
3,Date_next_renewal,Date of the next contract renewal (DD/MM/YYYY).
4,Distribution_channel,Classifies the channel through which the polic...



Variables al diccionari però no al dataset: {'Date_start _contract'}
Variables al dataset però no al diccionari: {'Date_start_contract'}

Nombre de files duplicades exactes: 0
Nombre d'IDs únics: 53502
Percentatge d'IDs repetits (panel): 0.4931

Taula de valors nuls (top 10):


,missing_count,missing_pct
Date_lapse,70408,66.70
Length,10329,9.79
Type_fuel,1764,1.67
ID,0,0.00
N_claims_year,0,0.00
N_doors,0,0.00
Value_vehicle,0,0.00
Cylinder_capacity,0,0.00
Power,0,0.00
Year_matriculation,0,0.00


Variables amb >10% nuls: ['Date_lapse']

Rangs mínim i màxim de variables numèriques:


,min,max
ID,1.000,53502.000
Distribution_channel,0.000,1.000
Seniority,1.000,40.000
Policies_in_force,1.000,17.000
Max_policies,1.000,17.000
Max_products,1.000,4.000
Lapse,0.000,7.000
Payment,0.000,1.000
Premium,40.140,2993.340
Cost_claims_year,0.000,260853.240



--- RESUM INICIAL ---
Registres: 105555
Variables: 30
Duplicats exactes: 0
Variables amb >10% nuls: ['Date_lapse']


#### <b>3.3.1.2 Conversió i validació de variables temporals</b>

In [20]:
# ============================================================
# 3.3.1.2 Conversió i validació de variables temporals
# ============================================================
# En aquest punt assumeixo que el DataFrame `df` ja està creat i carregat.
# Aquí el que faig és:
#   - Convertir les columnes de data al tipus datetime de pandas
#   - Fer una validació bàsica de coherència temporal (ordre cronològic)
#   - Derivar algunes variables temporals simples que després em poden servir per analítica/modelatge
#   - Exportar el dataset resultant a CSV per tenir-ne una sortida neta i reutilitzable
# ============================================================

# ---------------------------------------------
# 1. Identificar variables temporals
# ---------------------------------------------
# Defineixo quines columnes haurien de ser dates. Ho deixo explícit perquè així controlo jo
# quines variables temporals tracto i evito convertir columnes que no toquen.
date_cols = [
    'Date_start_contract',  # Data d’inici de la pòlissa (quan comença la cobertura)
    'Date_last_renewal',    # Data de l’última renovació del contracte
    'Date_next_renewal',    # Data prevista per a la següent renovació
    'Date_birth',           # Data de naixement del titular/conductor
    'Date_driving_licence', # Data d’obtenció del carnet de conduir
    'Date_lapse'            # Data d’un lapse/sinistre o d’un punt temporal rellevant (segons definició del dataset)
]

# Ho imprimeixo per pantalla només per comprovar ràpidament que estic agafant les columnes correctes.
print("Variables temporals detectades:", date_cols)

# ---------------------------------------------
# 2. Conversió a datetime
# ---------------------------------------------
# Recorro cada columna i la passo a datetime. Em baso en el format dia/mes/any perquè és el que espero al fitxer.
# Si hi ha valors que no encaixen amb el format, prefereixo que quedin com a NaT (i així els detecto després).
for col in date_cols:
    # format='%d/%m/%Y' vol dir que espero dates tipus 31/12/2020 (dia/mes/any)
    # errors='coerce' fa que els valors mal formats es converteixin a NaT, en lloc de petar l'execució
    df[col] = pd.to_datetime(df[col], format='%d/%m/%Y', errors='coerce')

# Després de convertir, reviso que els tipus siguin els correctes.
print("\nTipus després de la conversió:")
print(df[date_cols].dtypes)  # L’ideal és que totes surtin com datetime64[ns]

# També compto quants NaT he generat (això em dona una pista de si hi ha problemes de format o valors estranys).
print("\nValors NaT després de la conversió:")
print(df[date_cols].isna().sum())  # Resum dels nuls per a cada variable temporal

# ---------------------------------------------
# 3. Validació temporal bàsica
# ---------------------------------------------
# Aquí faig comprovacions molt bàsiques d'ordre cronològic per detectar casos clarament incoherents.
# No arreglo res, només compto anomalies per saber si tinc un problema real amb les dates.

birth_anomaly = (df["Date_birth"] > df["Date_driving_licence"]).sum()
# Casos on la data de naixement és posterior a la d'obtenció del carnet (això és impossible, així que és un senyal d'error)

licence_anomaly = (df["Date_driving_licence"] > df["Date_last_renewal"]).sum()
# Casos on el carnet s'obté després de l'última renovació de la pòlissa (com a mínim és sospitós i ho vull tenir controlat)

contract_anomaly = (df["Date_start_contract"] > df["Date_last_renewal"]).sum()
# Casos on la pòlissa comença després de l'última renovació registrada (aquí normalment hi ha un error clar)

next_negative = (df["Date_next_renewal"] < df["Date_last_renewal"]).sum()
# Casos on la següent renovació és anterior a la darrera (inconsistència temporal directa)

lapse_before_contract = (df["Date_lapse"] < df["Date_start_contract"]).sum()
# Casos on el lapse/sinistre passa abans que comenci la pòlissa (si realment és un sinistre, no hauria de ser possible)

lapse_before_last_renewal = (df["Date_lapse"] < df["Date_last_renewal"]).sum()
# Casos on el lapse/sinistre és anterior a l’última renovació (això pot ser error o pot requerir interpretació del significat de 'lapse')

# Ho imprimeixo tot junt per tenir una fotografia ràpida de la qualitat temporal.
print("\n--- VALIDACIÓ TEMPORAL ---")
print("birth_anomaly:", birth_anomaly, "anomalies")
print("licence_anomaly:", licence_anomaly, "anomalies")
print("contract_anomaly:", contract_anomaly, "anomalies")
print("next_negative:", next_negative, "anomalies")
print("lapse_before_contract:", lapse_before_contract, "anomalies")
print("lapse_before_last_renewal:", lapse_before_last_renewal, "anomalies")

# ---------------------------------------------
# 4. Derivació de magnituds temporals bàsiques
# ---------------------------------------------
# A partir de les dates, creo variables numèriques (en anys o dies) que normalment són molt útils per models.
# Ho faig amb diferències de dates i ho passo a dies, i després a anys quan toca.

df["Driver_age"] = (df["Date_last_renewal"] - df["Date_birth"]).dt.days / 365.25
# Edat aproximada del conductor a la darrera renovació. Divideixo per 365.25 per tenir en compte anys de traspàs.

df["Licence_age"] = (df["Date_last_renewal"] - df["Date_driving_licence"]).dt.days / 365.25
# Antiguitat del carnet (en anys) a la darrera renovació.

df["Policy_age"] = (df["Date_last_renewal"] - df["Date_start_contract"]).dt.days / 365.25
# Antiguitat de la pòlissa (en anys) des de l'inici fins a la darrera renovació.

df["Days_to_next"] = (df["Date_next_renewal"] - df["Date_last_renewal"]).dt.days
# Dies fins a la següent renovació a partir de la darrera (aquí ho deixo en dies perquè és més directe).

# Miro estadístiques bàsiques per veure si surten valors absurds (negatius, edats exagerades, etc.).
print("\nDescripció estadística de les variables derivades:")
print(df[["Driver_age", "Licence_age", "Policy_age", "Days_to_next"]].describe())

# I també ensenyo unes quantes files per inspecció manual ràpida (dates + variables derivades).
print(df[[
    "Date_start_contract", "Date_last_renewal", "Date_next_renewal",
    "Date_birth", "Date_driving_licence", "Date_lapse",
    "Driver_age", "Licence_age", "Policy_age", "Days_to_next"
]].head())

# ---------------------------------------------
# 5. Exportació del dataset net
# ---------------------------------------------
# Exporto el DataFrame a CSV per deixar una sortida preparada per a passos posteriors.
df.to_csv("clean_motor_insurance.csv", index=False, encoding="utf-8")
# index=False evita que se m'afegeixi la columna d'índex al fitxer (no la vull al CSV final)
# encoding="utf-8" em garanteix que accents i caràcters especials es guardin bé

# Missatge final per confirmar que s'ha generat l'arxiu.
print("\nFitxer exportat: clean_motor_insurance.csv")


Variables temporals detectades: ['Date_start_contract', 'Date_last_renewal', 'Date_next_renewal', 'Date_birth', 'Date_driving_licence', 'Date_lapse']

Tipus després de la conversió:
Date_start_contract     datetime64[ns]
Date_last_renewal       datetime64[ns]
Date_next_renewal       datetime64[ns]
Date_birth              datetime64[ns]
Date_driving_licence    datetime64[ns]
Date_lapse              datetime64[ns]
dtype: object

Valors NaT després de la conversió:
Date_start_contract         0
Date_last_renewal           0
Date_next_renewal           0
Date_birth                  0
Date_driving_licence        0
Date_lapse              70408
dtype: int64

--- VALIDACIÓ TEMPORAL ---
birth_anomaly: 0 anomalies
licence_anomaly: 31 anomalies
contract_anomaly: 68 anomalies
next_negative: 0 anomalies
lapse_before_contract: 0 anomalies
lapse_before_last_renewal: 404 anomalies

Descripció estadística de les variables derivades:
          Driver_age    Licence_age     Policy_age   Days_to_next
cou